# Import things

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
random.seed(42)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Load data

In [2]:
red_wine = pd.read_csv("./resources/resources/winequality-red.csv", sep=';')

# V.1 Exploring the green reds

In [3]:
red_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
red_wine.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

## a)

In [5]:
def plot_scatter_matrix(wine_data, good_threshhold, bad_threshold, save_plot=False):
    """Visualizes the data
    """
    length = len(wine_data.columns)
    fig, ax = plt.subplots(length, length, figsize=(50, 50))
    bigger = wine_data.where(wine_data['quality'] > good_threshhold)
    smaller = wine_data.where(wine_data['quality'] < bad_threshold)
    
    for i in range(length):
        for j in range(length):
            if i == j:
                ax[(i, j)].text(0.5, 0.5, '\n'.join(wine_data.columns[i].split()), fontsize=35, ha='center', va='center')
            else:
                ax[(j, i)].plot(bigger.iloc[:, i], bigger.iloc[:, j], 'co')
                ax[(j, i)].plot(smaller.iloc[:, i], smaller.iloc[:, j], 'mo')
    if save_plot:
        plt.savefig('scatter_matrix.png')
# plot_scatter_matrix(red_wine, 6, 5)

In [6]:
# plot_scatter_matrix(red_wine, 7, 4)

## b)

I think it can be pH, sulphates, alcohol and total sulfur dioxide. Cause there
different
groups of wine(based on quality) r better splitted.

# V.2 Learning to perceptron

In [7]:
data_3_8 = red_wine[(red_wine['quality'] >= 8) | (red_wine['quality'] <= 3)].reset_index(drop=True)
y_3_8 = [1 if i == True else 0 for i in data_3_8['quality'] >= 8]

In [8]:
def display(ret, dot=100, detail=10000, final=False):
    """Displays text info about training process"""
    if final:
        print()
        print('DONE')
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
        print('final weights = ', ret[-1][2], ', bias = ', ret[-1][3])
    if ret[-1][0] == 0:
        return
    if ret[-1][0] % dot == 0:
        print('.', end='')
    if ret[-1][0] % detail == 0:
        print()
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
        print('weights change = ', ret[-1][2][0] - ret[-detail - 1][2][0], ret[-1][2][1] - ret[-detail - 1][2][1])
        print('weights = ', ret[-1][2], 'bias = ', ret[-1][3])
        print('bias change = ', ret[-1][3] - ret[-detail - 1][3])

Displays info when model is learning

## a), b)

In [9]:
class perceptron:
    """A simple perceptron, with randomly initialized weights and bias"""

    def __init__(self, amount):
        """Random data initialization"""
        random.seed(42)
        self.amount = amount
        self.weights = [random.random() for n in range(amount)]
        self.bias = random.random()
        for i in range(amount):
            print('Weights[', i, '] = ', self.weights[i], sep='')
        print('bias = ', self.bias, '\n')

    def activation(self, value):
        """Activation function"""
        return 1 if value > 0 else 0

    def predict(self, data, alpha = None, y = None):
        """Predicts answer and updates weights to fit the data item better
        Returns predicted value or whether he guessed right
        """
        rez = self.activation(sum([data[i] * self.weights[i]
                                   for i in range(self.amount)]) + self.bias)
        if y == None:
            return rez
        if rez != y:
            self.weights = [self.weights[i] + alpha * data[i] * (y - rez) 
                            for i in range(self.amount)]
            self.bias = self.bias + alpha * (y - rez)
        return rez != y

    def train(self, data, y, epochs = 0, alpha = 0.001):
        """Trains perceptron epoch times, and logs results"""
        ret = []
        errors = 1
        if epochs == 0:
            i = 0
            while (errors):
                errors = 0
                for item in data.itertuples():
                    errors += self.predict(item[1:], alpha, y[item[0]])
                ret.append([i, errors, self.weights, self.bias])
                i += 1
                display(ret)
            display(ret, final=True)
        else:
            for i in range(epochs):
                errors = 0
                for item in data.itertuples():
                    errors += self.predict(item[1:], alpha, y[item[0]])
                ret.append([i, errors, self.weights, self.bias])
                display(ret)
                if errors == 0:
                    break
            display(ret, final=True)
        return ret

In [10]:
model = perceptron(2)
logs = model.train(data_3_8[['pH', 'alcohol']], y_3_8, 0, 0.01)

Weights[0] = 0.6394267984578837
Weights[1] = 0.025010755222666936
bias =  0.27502931836911926 

....................................................................................................
cycle = 10000, errors = 2
weights change =  -7.069099999996053 1.848000000000045
weights =  [-6.536073201538169, 1.653510755222712] bias =  4.455029318369069
bias change =  4.209999999999949
...............................
DONE
cycle = 13154, errors = 0
final weights =  [-7.259373201536793, 1.731510755222712] , bias =  6.135029318369033


## c) && bonus V.2

In [11]:
def plot_performance(performance, wine_data, good_threshold, bad_threshold, epoch=-1, save_plot=False, debug=False):
    """Plot the performance of my perceptron"""
    if debug:
        fig, ax = plt.subplots(1, 2, figsize=(30, 30))
    else:
        fig, ax = plt.subplots(1, 2)#, figsize=(10, 10))
    bigger = wine_data.where(wine_data['quality'] > good_threshold).reset_index(drop=True)
    smaller = wine_data.where(wine_data['quality'] < bad_threshold).reset_index(drop=True)
    
    ax[0].plot([i[1] for i in performance], 'g')
    ax[0].set_title('Errors as a function of epoch')
    ax[0].set_ylabel('Classification errors')
    ax[0].set_xlabel('Epoch')
    ax[1].plot(bigger['alcohol'], bigger['pH'], 'mo', label = 'good wines (> ' + str(good_threshold) + ' score)')
    ax[1].plot(smaller['alcohol'], smaller['pH'], 'co', label = 'bad wines (< ' + str(bad_threshold) + ' score)')
    ax[1].set_title('Decision boundary on epoch:' + str(len(performance) if epoch == -1 else epoch))
    ax[1].set_xlabel('Alcohol')
    ax[1].set_ylabel('pH')
    
    val = {}
    if debug:
        val['alcohol'] = [-100, 100]         
    else:
#         val['alcohol'] = [5, 15]         
        val['alcohol'] = [min(wine_data['alcohol']) - 1, max(wine_data['alcohol']) + 1] 
    val['pH'] = [
            - (performance[epoch][2][1] * val['alcohol'][0] + performance[epoch][3]) / performance[epoch][2][0],
            - (performance[epoch][2][1] * val['alcohol'][1] + performance[epoch][3]) / performance[epoch][2][0]
        ]
    ax[1].plot(val['alcohol'], val['pH'], 'g--')
    if debug:
        ax[1].set_xlim(-100, 100)
        ax[1].set_ylim(-100, 100)
        ax[1].fill_between(val['alcohol'], val['pH'], -100, facecolor='#f8bbd0')
        ax[1].fill_between(val['alcohol'], val['pH'], 100, facecolor='#b2ebf2')
    else:
#         ax[1].set_xlim(5, 15)
#         ax[1].set_ylim(2, 5)
        ax[1].fill_between(val['alcohol'], val['pH'], -5, facecolor='#f8bbd0')
        ax[1].fill_between(val['alcohol'], val['pH'], 15, facecolor='#b2ebf2')
        ax[1].set_xlim(min(wine_data['alcohol']) - 1, max(wine_data['alcohol']) + 1)
        ax[1].set_ylim(min(wine_data['pH']) - 1, max(wine_data['pH']) + 1)
#         ax[1].fill_between(val['alcohol'], val['pH'], min(wine_data['pH']), facecolor='#b2ebf2')
#         ax[1].fill_between(val['alcohol'], val['pH'], max(wine_data['pH']), facecolor='#f8bbd0')
    plt.tight_layout()
    plt.legend(loc='upper center', bbox_to_anchor=(1.6, 1))
    plt.show()

In [12]:
interactive(plot_performance, performance=fixed(logs), wine_data=fixed(red_wine), good_threshold=(0, 10),\
            bad_threshold=(0, 10), epoch=(0, len(logs) - 1))

interactive(children=(IntSlider(value=5, description='good_threshold', max=10), IntSlider(value=5, description…

## d) Feature scaling

In [13]:
normalized_red_wine = red_wine.copy()
for title in red_wine.columns:
    if not title == 'quality':
        normalized_red_wine[title] = (red_wine[title] - min(red_wine[title])) / (max(red_wine[title]) - min(red_wine[title]))
normalized_data_3_8 = normalized_red_wine[(red_wine['quality'] >= 8) | (red_wine['quality'] <= 3)].reset_index(drop=True)

In [14]:
model_normalized = perceptron(2)
logs_normalized = model_normalized.train(normalized_data_3_8[['pH', 'alcohol']], y_3_8, 0, 0.01)

Weights[0] = 0.6394267984578837
Weights[1] = 0.025010755222666936
bias =  0.27502931836911926 


DONE
cycle = 85, errors = 0
final weights =  [-0.09521887083345507, 0.10070306291497405] , bias =  0.015029318369119145


In [15]:
interactive(plot_performance, performance=fixed(logs_normalized), wine_data=fixed(normalized_red_wine), good_threshold=(0, 10),\
            bad_threshold=(0, 10), epoch=(0, len(logs_normalized) - 1))

interactive(children=(IntSlider(value=5, description='good_threshold', max=10), IntSlider(value=5, description…

# V.3 My fair ADALINE

## a)

Using stochastic learning algorithm instead of batch. Not adding polynomial features and not using more complicated algorhithms. No parallelism. Using only labels in learning, not values.

## b), c)

In [16]:
class adaline:
    """A simple adaline, with randomly initialized weights and bias"""

    def __init__(self, amount):
        """Random data initialization"""
        self.amount = amount
        self.weights = [random.random() for n in range(amount)]
        self.bias = random.random()

        
    def quantizer(self, value):
        """Quantizer function"""
        return 1 if value > 5.6 else 0

    
    def predict(self, data, y=False):
        """Predicts answer and updates weights to fit the data item better
        Returns predicted value or whether he guessed right
        """
        rez = sum([data[i] * self.weights[i] for i in range(self.amount)]) + self.bias
        if y == False:
            return self.quantizer(rez)
        else:
            err = y - rez
#             print('y =', y, 'rez =', rez, 'y - rez =', y - rez)
            return [[x * err for x in [*data, 1]], err]
        
    def train(self, data, y, epochs = 0, alpha = 0.001):
        """Trains perceptron epoch times, and logs results"""
        ret = []
        if epochs == 0:
            i = 0
            while (True):
                error = 0
                delta = [0] * (self.amount + 1)
                for item in data.itertuples():
                    [rez, err] = self.predict(item[1:], y[item[0]])
                    delta = [delta[i] + rez[i] for i in range(self.amount + 1)]
                    error += err
                self.weights = [self.weights[i] + alpha * delta[i] / len(data) for i in range(self.amount)]
                self.bias += alpha * delta[-1] / len(data)
                ret.append([i, error, self.weights, self.bias - 5])
                display(ret)
                if i > 0 and abs(ret[-1][1] - error) < 0.0000000001:
                    break
                i += 1
            display(ret, final=True)
        else:
            for i in range(epochs):
                error = 0
                delta = [0] * (self.amount + 1)
                for item in data.itertuples():
                    [rez, err] = self.predict(item[1:], y[item[0]])
                    delta = [delta[i] + rez[i] for i in range(self.amount + 1)]
                    error += err ** 2
#                 print(delta, errors)
                self.weights = [self.weights[i] + alpha * delta[i] / len(data) for i in range(self.amount)]
                self.bias += alpha * delta[-1] / len(data)
                ret.append([i, error, self.weights, self.bias - 5.6])
                display(ret)
                if i > 0 and abs(ret[-2][1] - error) < 0.0000000001:
                    print(ret[-1], error)
                    break
            display(ret, final=True)
        return ret

In [17]:
data_4_7 = red_wine[(red_wine['quality'] >= 7) | (red_wine['quality'] <= 4)].reset_index(drop=True)
y_4_7 = data_4_7['quality']

In [18]:
model = adaline(2)
adaline_logs = model.train(data_4_7[['pH', 'alcohol']], y_4_7.tolist(), 30000, 0.001)

....................................................................................................
cycle = 10000, errors = 376.3190315150191
weights change =  -0.8404198411323816 -0.004786962845589771
weights =  [-0.6283195699388097, 0.6941690043010255] bias =  -4.965177617498963
bias change =  -0.03854816030050223
....................................................................................................
cycle = 20000, errors = 370.32087159529937
weights change =  -0.2892037488245931 0.07534763244576803
weights =  [-0.9175233187634028, 0.7695166367467935] bias =  -4.860253452933253
bias change =  0.10492416456570997
...................................................................................................
DONE
cycle = 29999, errors = 368.1941398158499
final weights =  [-1.0436300088856048, 0.7937042475813745] , bias =  -4.716316565374624


In [19]:
interactive(plot_performance, performance=fixed(adaline_logs), wine_data=fixed(data_4_7), good_threshold=(0, 10),\
            bad_threshold=(0, 10), epoch=(0, len(adaline_logs) - 1), debug=False)

interactive(children=(IntSlider(value=5, description='good_threshold', max=10), IntSlider(value=5, description…

In [20]:
normalized_data_4_7 = normalized_red_wine[(red_wine['quality'] >= 7) | (red_wine['quality'] <= 4)].reset_index(drop=True)

In [21]:
model = adaline(2)
adaline_logs_normalized = model.train(normalized_data_4_7[['pH', 'alcohol']], y_4_7.tolist(), 30000, 0.1)

..................................................................................[8282, 329.9624293840112, [-4.001871819199307, 4.712259300674113], 0.5024108813615573] 329.9624293840112

DONE
cycle = 8282, errors = 329.9624293840112
final weights =  [-4.001871819199307, 4.712259300674113] , bias =  0.5024108813615573


In [22]:
pred = [model.predict(normalized_data_4_7.loc[i, ['pH', 'alcohol']]) for i in range(len(normalized_data_4_7))]
y = [1 if i == True else 0 for i in normalized_data_4_7['quality'] >= 7]

print(sum([abs((y[i] - pred[i])) for i in range(len(y))]), '/', len(normalized_data_4_7))

39 / 280


In [23]:
interactive(plot_performance, performance=fixed(adaline_logs_normalized), wine_data=fixed(normalized_data_4_7), good_threshold=(0, 10),\
            bad_threshold=(0, 10), epoch=(0, len(adaline_logs_normalized) - 1), debug=False)

interactive(children=(IntSlider(value=5, description='good_threshold', max=10), IntSlider(value=5, description…

# V.4

# a)

In [24]:
def holdout(input_data, percentage=30):
    data = input_data.sample(frac=1).reset_index(drop=True)
    sep = int(len(data) * percentage / 100)
    return (data[:sep], data[sep:])

In [25]:
train, test = holdout(normalized_data_4_7)
print(len(normalized_data_4_7), len(train), len(test))

280 84 196


# b)

In [26]:
def k_folds(input_data, k=10, shuffle=False):
    data = input_data.sample(frac=1).reset_index(drop=True) if shuffle else input_data
    seps = [int(len(data) / k) * i for i in range(k + 1)]
    parts = []
    for i in range(k):
        parts.append(data[seps[i]:seps[i + 1]])
    ret = []
    for i in range(k):
        train = pd.concat([parts[j] for j in range(k) if not j == i]).reset_index()
        test = parts[i].reset_index()
        ret.append((train, test))
    return ret

In [27]:
data = k_folds(normalized_data_4_7)
for train, test in data:
    print(len(normalized_data_4_7), len(train), len(test))

280 252 28
280 252 28
280 252 28
280 252 28
280 252 28
280 252 28
280 252 28
280 252 28
280 252 28
280 252 28


# c)

In [28]:
def cross_validate(input_data, k=10, epochs=10000, alpha=0.01):
    data = k_folds(input_data)
    error = []
    logs = []
    for train, test in data:
        model = adaline(2)
        log = model.train(train[['pH', 'alcohol']], train['quality'].tolist(), epochs, alpha)
        logs.append(log)
        y = [1 if i == True else 0 for i in train['quality'] >= 7]
        pred = [model.predict(train.loc[i, ['pH', 'alcohol']]) for i in range(len(train))]
        error.append(sum([abs(y[i] - pred[i]) for i in range(len(y))]))
    return (sum(error) / k)

When adaline trains it predicts continious value. It compares 4 vs 5 as example while we're inspecting model we compare binary values (1 vs 0). That's why values differ

In [29]:
cross_validate(normalized_data_4_7, epochs=1000)

.........
DONE
cycle = 999, errors = 393.0261194159426
final weights =  [1.1821076861292337, 2.6053538603261583] , bias =  -0.8813514640286391
.........
DONE
cycle = 999, errors = 432.11925917258105
final weights =  [1.0911196627093578, 2.5194950134024348] , bias =  -0.9088844334766106
.........
DONE
cycle = 999, errors = 444.4833581032538
final weights =  [1.5268554568414145, 2.4372932260027147] , bias =  -1.069997115020084
.........
DONE
cycle = 999, errors = 422.93475327930724
final weights =  [1.7161127094111663, 2.2104733314604474] , bias =  -0.9395797951096254
.........
DONE
cycle = 999, errors = 431.94518895415655
final weights =  [1.7095856853008768, 2.618144986749313] , bias =  -1.1371382480905448
.........
DONE
cycle = 999, errors = 480.24718238365745
final weights =  [1.8685604347011997, 2.606599747435001] , bias =  -1.2671831262366844
.........
DONE
cycle = 999, errors = 437.864285045972
final weights =  [1.046045926254339, 2.9337660395286465] , bias =  -1.0654673362965692


53.4

# V.5 Adventures in the Nth dimension

## a) && bonus V.1

In [30]:
%%cython
import pandas as pd
from libc.stdlib cimport malloc, free, rand, srand, RAND_MAX
from cpython cimport array
import array

red_wine = pd.read_csv("./resources/resources/winequality-red.csv", sep=';')

data_4_7 = red_wine[(red_wine['quality'] >= 7) | (red_wine['quality'] <= 4)].reset_index()
y_4_7 = data_4_7['quality'].tolist()
normalized_data_4_7 = data_4_7.copy()
for column in normalized_data_4_7.columns:
    if column != 'quality':
        normalized_data_4_7[column] = (data_4_7[column] - min(data_4_7[column])) / (max(data_4_7[column]) - min(data_4_7[column]))
data = normalized_data_4_7[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']].values.tolist()
        
def display(ret, dot=100, detail=10000, final=False):
    """Displays text info about training process"""
    if final:
        print()
        print('DONE')
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
    if ret[-1][0] == 0:
        return
    if ret[-1][0] % dot == 0:
        print('.', end='')
    if ret[-1][0] % detail == 0:
        print()
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
        
cdef struct s_perceptron:
    double *weights;
    double bias;
    int amount;
ctypedef s_perceptron t_perceptron

cdef t_perceptron init_perceptron(int amount):
    srand(42)
    cdef t_perceptron ret
    ret.amount = amount
    ret.weights = <double *>malloc(sizeof(double) * amount)
#     ret.bias = <double>rand() / RAND_MAX
    ret.bias = 0.27502931836911926
    ret.weights[0] = 0.6394267984578837
    ret.weights[1] = 0.025010755222666936
#     for i in range(amount):
#         ret.weights[i] = <double>rand() / RAND_MAX
    return ret
    
cdef int quantizer(double value):
    return 1 if value > 5.6 else 0
    
cdef list predict(t_perceptron *model, list data, int y):
    cdef double rez = model.bias
    for i in range(model.amount):
        rez += data[i] * model.weights[i]
    if y == -1:
        return [quantizer(rez)]
    cdef list ret = []
    cdef double err
    if rez != y:
        err = y - rez
        for i in range(model.amount):
            ret.append(data[i] * err)
        ret.append(err)
    return ret

cdef list train(t_perceptron *model, list data, list y, int epochs, double alpha):
    cdef list ret = []
    cdef unsigned long i = 0
    cdef list delta
    cdef double error = 1
    if epochs == 0:
        while (True):
            error = 0
            delta = [0] * (model.amount + 1)
            for j, item in enumerate(data):
                rez = predict(model, item, y[j])
                for k in range(model.amount + 1):
                    delta[k] += rez[k]
                error += rez[-1] ** 2
            for k in range(model.amount):
                model.weights[k] += alpha * delta[k] / len(data)
            model.bias += alpha * delta[-1] / len(data)
            ret.append([i, error, [model.weights[i] for i in range(model.amount)],
                        model.bias - 5.6])
            if i > 0 and abs(ret[-2][1] - error) < 0.0000000001:
                break
            i += 1
            display(ret)
        display(ret, final=True)
    else:
        for i in range(epochs):
            error = 0
            delta = [0] * (model.amount + 1)
            for j, item in enumerate(data):
                rez = predict(model, item, y[j])
                for k in range(model.amount + 1):
                    delta[k] += rez[k]
                error += rez[-1] ** 2
            for k in range(model.amount):
                model.weights[k] += alpha * delta[k] / len(data)
            model.bias += alpha * delta[-1] / len(data)
            ret.append([i, error, [model.weights[i] for i in range(model.amount)],
                        model.bias - 5.6])
            display(ret)
            if i > 0 and abs(ret[-2][1] - error) < 0.0000000001:
                break
        display(ret, final=True)
    return ret

cdef t_perceptron model = init_perceptron(len(data[0]))
train(&model, data, y_4_7, 1000000, 0.001)

pred = [predict(&model, data[i], -1) for i in range(len(data))]
      
y = [1 if i == True else 0 for i in normalized_data_4_7['quality'] >= 7]

print(sum([abs((y[i] - pred[i][0])) for i in range(len(y))]), '/', len(normalized_data_4_7))
free(model.weights)

....................................................................................................
cycle = 10000, errors = 251.48320672233177
weights change =  0.8650463175650619 -3.1352193064615763
weights =  [1.4606386452854836, -2.419700162349958, 0.780622025578617, -0.3846644806468307, -0.9119778929294038, 0.5202652125570416, -0.305392952427557, -0.39789054263478746, 0.6145963864552819, 2.153337179790007, 3.520473708010793] bias =  -1.4692799127790446
bias change =  3.710359287266349
....................................................................................................
cycle = 20000, errors = 236.93240073088967
weights change =  -0.012849754707971783 -0.702157034756512
weights =  [1.4477888905775118, -3.12185719710647, 0.09268209151867608, -0.5195233421844885, -1.5574192826342521, 0.6108005248942, -0.5853994691999379, -0.2729343254655107, -0.027957246119080285, 2.316622099847868, 3.462988536178477] bias =  -0.7375753657923907
bias change =  0.7317045469866539
......

It converges when data is separable in that hyperplane

## b)

It's called a hyperplane. For 2-dimensional space it's a point, for 3-d it's line and so on.

# V.6 Marvin's rebuttal && bonus V.1

In [33]:
%load_ext Cython

In [31]:
%%cython
import pandas as pd
from libc.stdlib cimport malloc, free, rand, srand, RAND_MAX
from cpython cimport array
import array

pan_gal = pd.read_csv("./resources/resources/Pan Galactic Gargle Blaster.csv", sep=';')

y_4_7 = pan_gal['quality'].tolist()
normalized_data_4_7 = data_4_7.copy()
for column in normalized_data_4_7.columns:
    if column != 'quality':
        normalized_data_4_7[column] = (data_4_7[column] - min(data_4_7[column])) / (max(data_4_7[column]) - min(data_4_7[column]))
data = normalized_data_4_7[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']].values.tolist()
        
def process_to_poly(poly):
    length = len(poly[0])
    for line in poly:
        for i in range(length):
            for j in range(length):
                line.append(line[i] * line[j])
        for i in range(length):
            for j in range(length):
                for k in range(length):
                    line.append(line[i] * line[j] * line[k])
    print('Was:', length, 'Now:', len(poly[0]))
    return poly

data = process_to_poly(data)

def display(ret, dot=100, detail=10000, final=False):
    """Displays text info about training process"""
    if final:
        print()
        print('DONE')
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
    if ret[-1][0] == 0:
        return
    if ret[-1][0] % dot == 0:
        print('.', end='')
    if ret[-1][0] % detail == 0:
        print()
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
        
cdef struct s_perceptron:
    double *weights;
    double bias;
    int amount;
ctypedef s_perceptron t_perceptron

cdef t_perceptron init_perceptron(int amount):
    srand(42)
    cdef t_perceptron ret
    ret.amount = amount
    ret.weights = <double *>malloc(sizeof(double) * amount)
#     ret.bias = <double>rand() / RAND_MAX
    ret.bias = 0.27502931836911926
    ret.weights[0] = 0.6394267984578837
    ret.weights[1] = 0.025010755222666936
#     for i in range(amount):
#         ret.weights[i] = <double>rand() / RAND_MAX
    return ret
    
cdef int quantizer(double value):
    return 1 if value > 5.6 else 0
    
cdef list predict(t_perceptron *model, list data, int y):
    cdef double rez = model.bias
    for i in range(model.amount):
        rez += data[i] * model.weights[i]
    if y == -1:
        return [quantizer(rez)]
    cdef list ret = []
    cdef double err
    if rez != y:
        err = y - rez
        for i in range(model.amount):
            ret.append(data[i] * err)
        ret.append(err)
    return ret

cdef list train(t_perceptron *model, list data, list y, int epochs, double alpha):
    cdef list ret = []
    cdef unsigned long i = 0
    cdef list delta
    cdef double error = 1
    if epochs == 0:
        while (True):
            error = 0
            delta = [0] * (model.amount + 1)
            for j, item in enumerate(data):
                rez = predict(model, item, y[j])
                for k in range(model.amount + 1):
                    delta[k] += rez[k]
                error += rez[-1] ** 2
            for k in range(model.amount):
                model.weights[k] += alpha * delta[k] / len(data)
            model.bias += alpha * delta[-1] / len(data)
            ret.append([i, error, [model.weights[i] for i in range(model.amount)],
                        model.bias - 5.6])
            if i > 0 and abs(ret[-2][1] - error) < 0.0000000001:
                break
            i += 1
            display(ret)
        display(ret, final=True)
    else:
        for i in range(epochs):
            error = 0
            delta = [0] * (model.amount + 1)
            for j, item in enumerate(data):
                rez = predict(model, item, y[j])
                for k in range(model.amount + 1):
                    delta[k] += rez[k]
                error += rez[-1] ** 2
            for k in range(model.amount):
                model.weights[k] += alpha * delta[k] / len(data)
            model.bias += alpha * delta[-1] / len(data)
            ret.append([i, error, [model.weights[i] for i in range(model.amount)],
                        model.bias - 5.6])
            display(ret)
            if i > 0 and abs(ret[-2][1] - error) < 0.0000000001:
                break
        display(ret, final=True)
    return ret

cdef t_perceptron model = init_perceptron(len(data[0]))
train(&model, data, y_4_7, 1000000, 0.001)

pred = [predict(&model, data[i], -1) for i in range(len(data))]
      
y = [1 if i == True else 0 for i in normalized_data_4_7['quality'] >= 5]

print(sum([abs((y[i] - pred[i][0])) for i in range(len(y))]), '/', len(normalized_data_4_7))
      
      
free(model.weights)

In [34]:
%%cython
import pandas as pd
from libc.stdlib cimport malloc, free, rand, srand, RAND_MAX
from cpython cimport array
import array

red_wine = pd.read_csv("./resources/resources/winequality-red.csv", sep=';')
data_4_7 = red_wine[(red_wine['quality'] >= 7) | (red_wine['quality'] <= 4)].reset_index()
y = [1 if i == True else 0 for i in data_4_7['quality'] >= 7]
normalized_data_4_7 = data_4_7.copy()
for column in normalized_data_4_7.columns:
    if column != 'quality':
        normalized_data_4_7[column] = (data_4_7[column] - min(data_4_7[column])) / (max(data_4_7[column]) - min(data_4_7[column]))
data = normalized_data_4_7[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']].values.tolist()

def process_to_poly(poly):
    length = len(poly[0])
    for line in poly:
        for i in range(length):
            for j in range(length):
                line.append(line[i] * line[j])
        for i in range(length):
            for j in range(length):
                for k in range(length):
                    line.append(line[i] * line[j] * line[k])
    print('Was:', length, 'Now:', len(poly[0]))
    return poly

# data = process_to_poly(data)

def display(ret, dot=100, detail=10000, final=False):
    """Displays text info about training process"""
    if final:
        print()
        print('DONE')
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
    if ret[-1][0] == 0:
        return
    if ret[-1][0] % dot == 0:
        print('.', end='')
    if ret[-1][0] % detail == 0:
        print()
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
        
cdef struct s_perceptron:
    double *weights;
    double bias;
    int amount;
ctypedef s_perceptron t_perceptron

cdef t_perceptron init_perceptron(int amount):
    srand(42)
    cdef t_perceptron ret
    ret.amount = amount
    ret.weights = <double *>malloc(sizeof(double) * amount)
    ret.bias = <double>rand() / RAND_MAX
#     ret.bias = 0.27502931836911926
#     ret.weights[0] = 0.6394267984578837
#     ret.weights[1] = 0.025010755222666936
    for i in range(amount):
        ret.weights[i] = <double>rand() / RAND_MAX
#         print('Weights[', i, '] = ', ret.weights[i], sep='')
#     print('bias = ', ret.bias, '\n')
    return ret
    
cdef int activation(double value):
    return 1 if value > 0 else 0
    
cdef double predict(t_perceptron *model, list data, double alpha, int y):
    cdef double rez = model.bias
    for i in range(model.amount):
        rez += data[i] * model.weights[i]
    rez = activation(rez)
    if y == -1:
        return rez
    if rez != y:
        for i in range(model.amount):
            model.weights[i] += alpha * data[i] * (y - rez)
        model.bias += alpha * (y - rez)
    return rez != y

cdef list train(t_perceptron model, list data, list y, int epochs, double alpha):
    cdef list ret = []
    cdef double errors = 1
    cdef unsigned long i = 0
    if epochs == 0:
        while (errors):
            errors = 0
            for j, item in enumerate(data):
                errors += predict(&model, item, alpha, y[j])
            ret.append([i, errors, [model.weights[i] for i in range(model.amount)],
                        model.bias])
            i += 1
            display(ret)
        display(ret, final=True)
    else:
        for i in range(epochs):
            errors = 0
            for j, item in enumerate(data):
                errors += predict(&model, item, alpha, y[j])
            ret.append([i, errors, [model.weights[i] for i in range(model.amount)],
                        model.bias])
            display(ret)
            if errors == 0:
                break
        display(ret, final=True)
    return ret

cdef t_perceptron model = init_perceptron(len(data[0]))
train(model, data, y, 100000, 0.00001)
free(model.weights)

....................................................................................................
cycle = 10000, errors = 44.0
....................................................................................................
cycle = 20000, errors = 38.0
....................................................................................................
cycle = 30000, errors = 35.0
....................................................................................................
cycle = 40000, errors = 25.0
....................................................................................................
cycle = 50000, errors = 24.0
....................................................................................................
cycle = 60000, errors = 28.0
....................................................................................................
cycle = 70000, errors = 26.0
..........................................................................................

In [35]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [37]:
%%cython
import pandas as pd
from libc.stdlib cimport malloc, free, rand, srand, RAND_MAX
from cpython cimport array
import array

red_wine = pd.read_csv("./resources/resources/winequality-red.csv", sep=';')

data_4_7 = red_wine[(red_wine['quality'] >= 7) | (red_wine['quality'] <= 4)].reset_index()
y_4_7 = data_4_7['quality'].tolist()
normalized_data_4_7 = data_4_7.copy()
for column in normalized_data_4_7.columns:
    if column != 'quality':
        normalized_data_4_7[column] = (data_4_7[column] - min(data_4_7[column])) / (max(data_4_7[column]) - min(data_4_7[column]))
data = normalized_data_4_7[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']].values.tolist()
        
def process_to_poly(poly):
    length = len(poly[0])
    for line in poly:
        for i in range(length):
            for j in range(length):
                line.append(line[i] * line[j])
        for i in range(length):
            for j in range(length):
                for k in range(length):
                    line.append(line[i] * line[j] * line[k])
    print('Was:', length, 'Now:', len(poly[0]))
    return poly

# data = process_to_poly(data)

def display(ret, dot=100, detail=10000, final=False):
    """Displays text info about training process"""
    if final:
        print()
        print('DONE')
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
    if ret[-1][0] == 0:
        return
    if ret[-1][0] % dot == 0:
        print('.', end='')
    if ret[-1][0] % detail == 0:
        print()
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
        
cdef struct s_perceptron:
    double *weights;
    double bias;
    int amount;
ctypedef s_perceptron t_perceptron

cdef t_perceptron init_perceptron(int amount):
    srand(42)
    cdef t_perceptron ret
    ret.amount = amount
    ret.weights = <double *>malloc(sizeof(double) * amount)
#     ret.bias = <double>rand() / RAND_MAX
    ret.bias = 0.27502931836911926
    ret.weights[0] = 0.6394267984578837
    ret.weights[1] = 0.025010755222666936
#     for i in range(amount):
#         ret.weights[i] = <double>rand() / RAND_MAX
    return ret
    
cdef int quantizer(double value):
    return 1 if value > 5.6 else 0
    
cdef list predict(t_perceptron *model, list data, int y):
    cdef double rez = model.bias
    for i in range(model.amount):
        rez += data[i] * model.weights[i]
    if y == -1:
        return [quantizer(rez)]
    cdef list ret = []
    cdef double err
    if rez != y:
        err = y - rez
        for i in range(model.amount):
            ret.append(data[i] * err)
        ret.append(err)
    return ret

cdef list train(t_perceptron *model, list data, list y, int epochs, double alpha):
    cdef list ret = []
    cdef unsigned long i = 0
    cdef list delta
    cdef double error = 1
    if epochs == 0:
        while (True):
            error = 0
            delta = [0] * (model.amount + 1)
            for j, item in enumerate(data):
                rez = predict(model, item, y[j])
                for k in range(model.amount + 1):
                    delta[k] += rez[k]
                error += rez[-1] ** 2
            for k in range(model.amount):
                model.weights[k] += alpha * delta[k] / len(data)
            model.bias += alpha * delta[-1] / len(data)
            ret.append([i, error, [model.weights[i] for i in range(model.amount)],
                        model.bias - 5.6])
            if i > 0 and abs(ret[-2][1] - error) < 0.0000000001:
                break
            i += 1
            display(ret)
        display(ret, final=True)
    else:
        for i in range(epochs):
            error = 0
            delta = [0] * (model.amount + 1)
            for j, item in enumerate(data):
                rez = predict(model, item, y[j])
                for k in range(model.amount + 1):
                    delta[k] += rez[k]
                error += rez[-1] ** 2
            for k in range(model.amount):
                model.weights[k] += alpha * delta[k] / len(data)
            model.bias += alpha * delta[-1] / len(data)
            ret.append([i, error, [model.weights[i] for i in range(model.amount)],
                        model.bias - 5.6])
            display(ret)
            if i > 0 and abs(ret[-2][1] - error) < 0.0000000001:
                break
        display(ret, final=True)
    return ret

cdef t_perceptron model = init_perceptron(len(data[0]))
train(&model, data, y_4_7, 1000000, 0.001)

pred = [predict(&model, data[i], -1) for i in range(len(data))]
      
y = [1 if i == True else 0 for i in normalized_data_4_7['quality'] >= 7]

print(sum([abs((y[i] - pred[i][0])) for i in range(len(y))]), '/', len(normalized_data_4_7))
      
      
free(model.weights)

....................................................................................................
cycle = 10000, errors = 343.0636037959219
....................................................................................................
cycle = 20000, errors = 309.90707869535083
....................................................................................................
cycle = 30000, errors = 292.60522723089235
....................................................................................................
cycle = 40000, errors = 281.6805991251494
....................................................................................................
cycle = 50000, errors = 273.79049565042914
....................................................................................................
cycle = 60000, errors = 267.630764373475
....................................................................................................
cycle = 70000, errors = 262.61589847789

....................................................................................................
cycle = 580000, errors = 225.95575289044297
....................................................................................................
cycle = 590000, errors = 225.87829532642638
....................................................................................................
cycle = 600000, errors = 225.80378142255256
....................................................................................................
cycle = 610000, errors = 225.7320642409571
....................................................................................................
cycle = 620000, errors = 225.66300725083045
....................................................................................................
cycle = 630000, errors = 225.59648337384974
....................................................................................................
cycle = 640000, errors = 225.5